In [1]:
import os
import gc

import pickle
import torch
import optuna
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset
from torchmetrics.classification import MultilabelAccuracy

from torchvision.models import resnet50
from torchvision.datasets import CIFAR10

/home/eirikmv/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class PennFudanDataset(Dataset):
    def __init__(self, images, labels, transforms):
        
        self.transforms = transforms

        self.images = images
        self.labels = labels        
        self.masks = None

    def __getitem__(self, idx):
        
        img = self.images[idx]
        label = self.labels[idx]

        
        # split the color-encoded mask into a set
        # of binary masks
        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)

        # get bounding box coordinates for each mask
        boxes = masks_to_boxes(masks)

        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = idx
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Wrap sample and targets into torchvision tv_tensors:
        img = tv_tensors.Image(img)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=F.get_size(img))
        target["masks"] = tv_tensors.Mask(masks)
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [3]:
# transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.RandomRotation(degrees=45),
#     transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

class Xray_dataset(Dataset):
    
    def __init__(self, images, labels, test : bool):

        self.images = images
        self.labels = labels
        self.test = test

        self.labels = torch.from_numpy(np.array(self.labels).astype(np.int32))
        
        self.transform_train = transforms.Compose([transforms.ToTensor(),
                                                   transforms.RandomRotation(20),
                                                   transforms.RandomHorizontalFlip()])

        self.transform_test = transforms.Compose([transforms.ToTensor()])

        self.transform = transforms.Compose([transforms.ToTensor()])


    def __len__(self):
            return len(self.images)
    
    def __getitem__(self, idx):
        
        img = self.images[idx]
        label = self.labels[idx]

        if not self.test:
            img = self.transform_train(img)

        elif self.test:
            img = self.transform_test(img)
        
        return img, label


In [4]:
data_dir = "/home/eirikmv/data/chestxray8/dataset"
batch_size = 64

train_x, train_y = pickle.load(open(os.path.join(data_dir, "train.pkl"), "rb"))
val_x, val_y = pickle.load(open(os.path.join(data_dir, "validation.pkl"), "rb"))

print(f"train size: {len(train_y)}") 
print(f"val size: {len(val_y)}")

train_dataloader = DataLoader(Xray_dataset(train_x, train_y, test = False), 
                                batch_size=batch_size, 
                                shuffle=True)

val_dataloader = DataLoader(Xray_dataset(val_x, val_y, test = False), 
                            batch_size=batch_size, 
                            shuffle=True)


train size: 33881
val size: 5767


In [9]:
#resnet50(pretrained=True)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr = 0.001
epochs = 50

model = resnet50(pretrained=True).to(device)
model.fc = nn.Linear(2048, len(train_y[0])).to(device)

criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

all_loss = []
all_train_acc = []
all_val_acc = []

print("Training model....")
for epoch in range(epochs):

    for (x, y) in tqdm(train_dataloader):
        
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        
        yhat = model(x)
        
        loss = criterion(yhat, y.float())

        loss.backward()    
        optimizer.step()
    
    lr_scheduler.step()
    
    print(f"Epoch:{epoch} Loss:{loss.item()}")
    
os.system("rm -f model/model.pt")
torch.save(model, "model/model.pt")

/home/eirikmv/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/eirikmv/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training model....


100%|██████████| 530/530 [07:36<00:00,  1.16it/s]


Epoch:0 Loss:0.2570437788963318


100%|██████████| 530/530 [07:47<00:00,  1.13it/s]


...

Epoch:4 Loss:0.23896558582782745


100%|██████████| 530/530 [07:54<00:00,  1.12it/s]


Epoch:5 Loss:0.19230560958385468


 69%|██████▉   | 366/530 [05:31<02:28,  1.10it/s]


KeyboardInterrupt: 

In [1]:
!nvidia-smi

Mon Nov 27 14:03:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM3-32GB           On  | 00000000:B9:00.0 Off |                    0 |
| N/A   33C    P0              80W / 350W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--